In [36]:
import pandas as pd
import numpy as np
import re
import time
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

empl_roll = pd.DataFrame(pd.read_excel('/Users/arlisscoates/Documents/salesforce_reports_manipulations_files/CR Indexed Worker Roster - 07-01.xlsx'))
user_df = pd.DataFrame(pd.read_csv('/Users/arlisscoates/Documents/salesforce_reports_manipulations_files/ihm_users.csv'))
market_owners = pd.DataFrame(pd.read_csv('/Users/arlisscoates/Documents/salesforce_reports_manipulations_files/ihm_account_management.csv'))
workday = pd.DataFrame(pd.read_csv('usersChanges_2021_02_15_11_54.csv'))
revenue = pd.DataFrame(pd.read_csv('/Users/arlisscoates/Documents/salesforce_reports_manipulations_files/report1624978643517 - report1624978643517.csv'))
projects = pd.DataFrame(pd.read_csv('/Users/arlisscoates/Documents/salesforce_reports_manipulations_files/ihm_projects.csv', error_bad_lines=False))
newhr = pd.DataFrame(pd.read_csv('/Users/arlisscoates/Documents/salesforce_reports_manipulations_files/March iHeart Everyone HR Pull.xlsx - Sheet1.csv'))
accounts = pd.DataFrame(pd.read_csv('/Users/arlisscoates/Documents/salesforce_reports_manipulations_files/report1618865683080.csv'))['Account Name']
ids = pd.DataFrame(pd.read_csv('/Users/arlisscoates/Documents/salesforce_reports_manipulations_files/report1624395738435.csv'))
markets_spend = pd.DataFrame(pd.read_csv('/Users/arlisscoates/Documents/salesforce_reports_manipulations_files/iheart_markets_spend.csv'))
user_ids = pd.DataFrame(data={'salesforce_user': ['Arliss Coates', 'Ashley McCarthy', 'Mariana Trevino', 'Jenny Donheiser',
           'James Donahue'], 'salesforce_user_id': ['005f4000004vNz7', '005f4000002nHZR', '005f4000005XlFt', '005f4000004DvNw', '005f4000002sCJ5']})
product_ids = pd.DataFrame(pd.read_csv('/Users/arlisscoates/Documents/salesforce_reports_manipulations_files/report1626385418073.csv')).drop_duplicates(
    subset=['Product: Product Name'])
newmonday = pd.DataFrame(pd.read_excel('/Users/arlisscoates/Downloads/iHM_Usage_Reports_1626371425.xlsx'))
newmonday.columns = newmonday.iloc[1]
newmonday.drop(newmonday.index[:3],axis=0,inplace=True)
newmonday.drop(newmonday.index[153:],axis=0,inplace=True)
newmonday = newmonday[['Name','Owner']]

print('The shape of revenue is: ' + str(revenue.shape))

b'Skipping line 1989: expected 12 fields, saw 13\nSkipping line 5196: expected 12 fields, saw 17\nSkipping line 13544: expected 12 fields, saw 14\nSkipping line 14913: expected 12 fields, saw 13\nSkipping line 16569: expected 12 fields, saw 13\nSkipping line 17635: expected 12 fields, saw 14\nSkipping line 21689: expected 12 fields, saw 13\nSkipping line 22012: expected 12 fields, saw 13\n'


The shape of revenue is: (21776, 16)


In [37]:
print("Reading...")
start = time.time()

def Differences(hrnames, accountnames):
    li_diff = [i for i in hrnames + accountnames if i not in hrnames or i not in accountnames]
    missingfromhr = [i for i in hrnames + accountnames if i not in hrnames]
    missingfromaccountnames = [i for i in hrnames + accountnames if i not in accountnames]
    if len(missingfromaccountnames) != 0:
        return 'missing from Salesforce account names:', missingfromaccountnames
    else:
        return print('full agreement')

# altering field names of tables to match Salesforce fields

## new empl_roll
# monday_owners.columns = monday_owners.iloc[3]
# monday_owners.drop(monday_owners.index[:4],axis=0,inplace=True)

empl_roll.rename(columns = {'Worker': 'hr_worker', 'Region/Area': 'hr_region_area', 'Market': 'hr_market', 'Job Title': 'hr_job_title', 'Email - Primary Work': 'hr_email',
                        "Worker's Manager": 'hr_worker_manager', 'Hire Date': 'hr_hire_date'}, inplace=True)

user_df.rename(columns = {'client': 'adwire_client', 'group_x': 'adwire_group', 'name_x': 'adwire_name_x',
                                       'email': 'adwire_email', 'projects': 'adwire_projects', 'spend': 'adwire_spend',
                                       'project_ts': 'adwire_first_posting', 'group_cap': 'adwire_cap', 'owner': 'account_owner',
                                        'group_upper': 'adwire_group_upper'}, inplace=True)

workday.rename(columns = {'first name_x': 'workday_first_name', 
                                       'action': 'workday_action', 'client code': 'workday_client_code', 'first name_y': 'workday_first_name2',
                                       'last name': 'workday_last_name', 'group_y': 'workday_group_y'}, inplace=True)

revenue.rename(columns = {'Opportunity: Opportunity ID': 'iHeart Opportunity ID',
                            'Opportunity: Stage': 'Stage',
                           'Opportunity: Opportunity Owner: Market Division': 'Revenue Market Division',
                           'Opportunity: Opportunity Owner: Market Area': 'Revenue Market Area',
                           'Opportunity: Opportunity Owner: Home Operating Market': 'Account Name',
                           'Opportunity: Opportunity Owner: Full Name': 'iHeart Oppty Owner',
                           'OppOwnerEmail': 'email',
                           'New Business': 'new_business',
                           'Opportunity: Account Name: Account Name': 'opp_account_name',
                           'Opportunity: Opportunity Name': 'Opportunity Name',
                           'Adwire Project ID': 'project_id',
                           'Form Date': 'form_date',
                           'Opportunity: Created Date': 'Created Date',
                           'Opportunity: Close Date': 'Close Date',
                           'Opportunity: Gross Expected Revenue': 'Expected iHeart Revenue',
                           'Opportunity: Gross Amount': 'iHeart Gross Revenue',
                           'Form Created on Closed Opp': 'Created on Closed Opp',
                           'Opportunity: Opportunity Owner: Market Type': 'Revenue Market Type'}, inplace=True)

projects.rename(columns = {'project_id': 'project_id', 'adwire_post_date': 'project_post_date', 'email': 'creator_email', 'title': 'project_name',
                           'name': 'Product', 'name.1': 'project_creator', 'create_ts': 'project_post_date', 'name_y': 'projects_name_y', 'group': 'project_group',
                                        'submissions': 'projects_submissions', 'opportynity_id': 'project_opportunity_id',
                                       'credits_used': 'project_credits_used', 'industry': 'project_industry', 'tone': 'project_tone'}, inplace=True)

print("Took {:.2f} seconds".format(time.time() - start))

Reading...
Took 0.05 seconds


#####

Add record type to revenue

In [38]:
print("Reading...")
start = time.time()

# Aligning market names to match Salesforce account names

newmonday.Name = ['iHeart Media - ' + x[:-3].title() + x[-3:].upper() for x in newmonday.Name]

newmonday.loc[(newmonday.Name == 'iHeart Media - Ihm Innovation LAB'), 'Name'] = 'iHeart Media - IHM Innovation Lab'
newmonday.loc[(newmonday.Name == 'iHeart Media - Corporate GrOUP'), 'Name'] = 'iHeart Media - Corporate Group'
newmonday.loc[(newmonday.Name == 'iHeart Media - Inside SaLES'), 'Name'] = 'iHeart Media - Inside Sales'
newmonday.loc[(newmonday.Name == 'iHeart Media - Mcallen, TX'), 'Name'] = 'iHeart Media - McAllen, TX'
newmonday.loc[(newmonday.Name == 'iHeart Media - Tallahassee, FL'), 'Name'] = 'iHeart Media - Panama City/Tallahassee, FL'

newmonday.loc[(newmonday.Owner == 'Ashley McCarthy, Jenny Donheiser'), 'Owner'] = 'Jenny Donheiser'
newmonday.loc[(newmonday.Owner == 'Ashley McCarthy, Mariana E Trevino Stafford'), 'Owner'] = 'Mariana Trevino'
newmonday.loc[(newmonday.Owner == 'Mariana E Trevino Stafford'), 'Owner'] = 'Mariana Trevino'
newmonday.loc[(newmonday.Owner == 'Arliss Coates, Jenny Donheiser'), 'Owner'] = 'Arliss Coates'

#############
#############

empl_roll.hr_email = empl_roll.hr_email.str.lower()

#############
#############

newhr['User - Division'] = ['iHeart Media - ' + x.replace(' - Market', '') for x in newhr['User - Division']]
newhr.loc[(newhr['User - Division'] == 'iHeart Media - Inside Sales & Bus Development'), 'User - Division'] = 'iHeart Media - Inside Sales'

print('newhr: ' + str(Differences(newhr['User - Division'].unique().tolist(), accounts.tolist())))

#############
#############

revenue['form_date'] = pd.to_datetime(revenue['form_date'], errors='coerce').dt.strftime('%m/%d/%Y')

revenue = revenue[(revenue.form_date > '02/19/2020')]
# revenue = revenue[(revenue.form_date < '04/28/2021')]

re_outer = re.compile(r'([^A-Z ])([A-Z])')
re_inner = re.compile(r'\b[A-Z]+(?=[A-Z][a-z])')
revenue['Account Name'] = ['iHeart Media - ' + re_inner.sub(r'\g<0> ', re_outer.sub(r'\1 \2', x)) for x in revenue['Account Name'].astype(str)]
revenue['Account Name'] = [x[:-3] + ', ' + x[-2:] for x in revenue['Account Name']]

revenue.loc[(revenue['Account Name'] == 'iHeart Media - St Petersburg, FL'),'Account Name'] = 'iHeart Media - Inside Sales'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Fort Myers, FL'), 'Account Name'] = 'iHeart Media - Fort Myers/Naples, FL'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Cedar Rapids, IA'), 'Account Name'] = 'iHeart Media - Cedar Rapids/Iowa City, IA'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Panama City, FL'), 'Account Name'] = 'iHeart Media - Panama City/Tallahassee, FL'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Tallahassee, FL'), 'Account Name'] = 'iHeart Media - Panama City/Tallahassee, FL'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Roanoke, VA'), 'Account Name'] = 'iHeart Media - Roanoke/Lynchburg, VA'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Sarasota, FL'), 'Account Name'] = 'iHeart Media - Sarasota/Bradenton, FL'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Ashland, OH'),'Account Name'] = 'iHeart Media - Ashland/Mansfield, OH'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - New York, NY'),'Account Name'] = 'iHeart Media - New York City, NY'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Modesto, CA'),'Account Name'] = 'iHeart Media - Modesto/Stockton, CA'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Hattiesburg, MS'),'Account Name'] = 'iHeart Media - Laurel, MS'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Albany, GA'),'Account Name'] = 'iHeart Media - Columbus, GA'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Mc Allen, TX'),'Account Name'] = 'iHeart Media - McAllen, TX'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - , an'),'Account Name'] = 'iHeart Media - No Data'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Ta, pa'),'Account Name'] = 'iHeart Media - Tampa, FL'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Bay Area, CA'),'Account Name'] = 'iHeart Media - San Francisco, CA'

revenue = pd.merge(revenue, ids[['Email', 'Account ID', 'Contact ID']], left_on='email', right_on='Email', how='left')

# Essentially a VLOOKUP; backfilling Account Name and Revenue Market Area fields with information from newhr because many of these values are consistently absent from
# the revenue report alone.

merge_newhr = newhr.rename(columns = {'User - User Email': 'email'})
merge_newhr['email'] = merge_newhr.email.str.lower()

revenue = revenue.merge(merge_newhr[['email', 'User - Division', 'User - Division Parent']], on='email', how='left')
revenue['Account Name'] = np.where(revenue['Account Name'] == 'iHeart Media - No Data', revenue['User - Division'], revenue['Account Name'])
revenue['Revenue Market Area'] = np.where(revenue['Revenue Market Area'].isnull(), revenue['User - Division Parent'], revenue['Revenue Market Area'])

revenue.drop('User - Division', axis=1, inplace=True)
revenue.drop('User - Division Parent', axis=1, inplace=True)

revenue = pd.merge(revenue, newmonday[['Name','Owner']], left_on='Account Name', right_on='Name', how='left')

print('revenue: ' + str(Differences(revenue['Account Name'].unique().tolist(), accounts.tolist())))
print(revenue.shape)

revenue['record_type'] = "0125G000000uFriQAE"

print('revenue: ' + str(Differences(revenue['Account Name'].unique().tolist(), accounts.tolist())))

#############
#############

user_df.group = ['iHeart Media - ' + x for x in user_df.group]
user_df.group = user_df.group.str.strip()
user_df.adwire_email = user_df.adwire_email.str.lower()

user_df.adwire_first_posting = pd.to_datetime(user_df.adwire_first_posting, errors='coerce').dt.strftime('%m/%d/%Y')

user_df = pd.merge(user_df, ids[['Email', 'Account ID', 'Contact ID', 'First Name', 'Last Name']], left_on='adwire_email', right_on='Email', how='left')

user_df = pd.merge(user_df, empl_roll, left_on='adwire_email', right_on='hr_email', how='right')

user_df = pd.merge(user_df, newmonday[['Name','Owner']], left_on='group', right_on='Name', how='left')

user_df['record_type'] = "0125G000000uFriQAE"

user_df.loc[(user_df.group == 'iHeart Media - St. Petersburg, FL | Inside Sales'), 'group'] = 'iHeart Media - Inside Sales'
user_df.loc[(user_df.group == 'iHeart Media - Lexington / Somerset, KY'), 'group'] = 'iHeart Media - Lexington/Somerset, KY'
user_df.loc[(user_df.group == 'iHeart Media - Roanoke / Lynchburg, VA'), 'group'] = 'iHeart Media - Roanoke/Lynchburg, VA'
user_df.loc[(user_df.group == 'iHeart Media - Modesto / Stockton, CA'), 'group'] = 'iHeart Media - Modesto/Stockton, CA'

print('user_df: ' + str(Differences(user_df.group.unique().tolist(), accounts.tolist())))

#############
#############

projects.project_post_date = pd.to_datetime(projects.project_post_date)
projects['project_post_date'] = pd.to_datetime(projects['project_post_date'], errors='coerce').dt.strftime('%Y/%m/%d')
# projects = projects[(projects.project_post_date > '02/19/2020')]
# projects = projects[(projects.project_post_date < '04/28/2021')]

projects['project_opportunity_id'] = projects['project_opportunity_id'].map(lambda x: str(x)[:-3])

projects = projects[projects.project_opportunity_id.map(len) == 15]
projects.creator_email = projects.creator_email.str.lower()

projects.project_group = ['iHeart Media - ' + x for x in projects.project_group]
projects.project_group = projects.project_group.str.strip()

projects = projects.loc[(projects.project_group != 'iHeart Media - 9') & (projects.project_group != 'iHeart Media - 6')]

projects.loc[(projects.project_group == 'iHeart Media - St. Petersburg, FL | Inside Sales'), 'project_group'] = 'iHeart Media - Inside Sales'
projects.loc[(projects.project_group == 'iHeart Media - Lexington / Somerset, KY'), 'project_group'] = 'iHeart Media - Lexington/Somerset, KY'
projects.loc[(projects.project_group == 'iHeart Media - Roanoke / Lynchburg, VA'), 'project_group'] = 'iHeart Media - Roanoke/Lynchburg, VA'
projects.loc[(projects.project_group == 'iHeart Media - Modesto / Stockton, CA'), 'project_group'] = 'iHeart Media - Modesto/Stockton, CA'

projects = pd.merge(projects, user_df[['adwire_email','adwire_first_posting', 'adwire_cap']], left_on = 'creator_email', right_on = 'adwire_email', how='left')
projects = pd.merge(projects, ids[['Email', 'Contact ID', 'Account ID']], left_on = 'creator_email', right_on = 'Email', how='left')
projects = pd.merge(projects,newmonday[['Name','Owner']],left_on='project_group',right_on='Name',how='left')

projects = pd.merge(projects,user_ids[['salesforce_user','salesforce_user_id']], left_on='Owner',right_on='salesforce_user',how='left')
projects = pd.merge(projects, product_ids[['Product: Product ID', 'Product: Product Name']], left_on='Product', right_on='Product: Product Name', how='left')

projects['record_type'] = "0125G000000uFriQAE"

#############
#############

accounts_ids = ids[['Account Name','Account ID']]
accounts_ids.drop_duplicates(subset='Account ID',inplace=True)
caps = pd.DataFrame(pd.read_csv('ihm_caps.csv'))
# caps = pd.merge(accounts_ids,caps,left_on='')
caps.group = ['iHeart Media - ' + x for x in caps.group]
caps.group = caps.group.str.strip()

caps = caps.loc[(caps.group != 'iHeart Media - 9') & (caps.group != 'iHeart Media - 6')]

caps.loc[(caps.group == 'iHeart Media - St. Petersburg, FL | Inside Sales'), 'group'] = 'iHeart Media - Inside Sales'
caps.loc[(caps.group == 'iHeart Media - Lexington / Somerset, KY'), 'group'] = 'iHeart Media - Lexington/Somerset, KY'
caps.loc[(caps.group == 'iHeart Media - Roanoke / Lynchburg, VA'), 'group'] = 'iHeart Media - Roanoke/Lynchburg, VA'
caps.loc[(caps.group == 'iHeart Media - Modesto / Stockton, CA'), 'group'] = 'iHeart Media - Modesto/Stockton, CA'

accounts_ids = pd.merge(accounts_ids, caps[['group','group_cap']], left_on='Account Name',right_on='group',how='left')

print('projects: ' + str(Differences(projects.project_group.unique().tolist(), accounts.tolist())))
print('projects lower limit: ' + projects.project_post_date.min(), 'projects upper limit: ' + projects.project_post_date.max())
print('revenue lower limit: ' + revenue.form_date.min(), 'revenue upper limit: ' + revenue.form_date.max())
print('revenue shape is: ' + str(revenue.shape))
print('the shape of projects is : ' + str(projects.shape))
print('market ownership: ' + str(newmonday.Owner.value_counts()))
print("Took {:.2f} seconds".format(time.time() - start))

Reading...
full agreement
newhr: None
revenue: ('missing from Salesforce account names:', [nan])
(19324, 21)
revenue: ('missing from Salesforce account names:', [nan])
user_df: ('missing from Salesforce account names:', ['iHeart Media - Unsorted Users', nan])
full agreement
projects: None
projects lower limit: 2018/06/26 projects upper limit: 2021/07/13
revenue lower limit: 02/19/2021 revenue upper limit: NaT
revenue shape is: (19324, 22)
the shape of projects is : (22696, 25)
market ownership: Jenny Donheiser    49
Mariana Trevino    48
Arliss Coates      48
Ashley McCarthy     5
Name: Owner, dtype: int64
Took 6.35 seconds


/Users/arlisscoates/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [50]:
noposters = user_df.loc[(user_df.adwire_email.str.contains('Account',na=False)) & (user_df.adwire_spend == '0.0')]

In [49]:
user_df.head(40)

,adwire_client,group,name,adwire_email,status,adwire_projects,adwire_spend,adwire_first_posting,adwire_cap,Email,Account ID,Contact ID,First Name,Last Name,hr_worker,hr_region_area,hr_market,hr_job_title,hr_email,hr_worker_manager,hr_hire_date,Name,Owner,record_type
0,iHeart Media,"iHeart Media - Akron, OH",Andrea Katusin,andreakatusin@iheartmedia.com,4.0,6.0,6.0,04/14/2020,10.0,andreakatusin@iheartmedia.com,001f400001KcDFz,003f400001PRfjU,Andrea,Katusin,Andrea Katusin,Cleveland - Area,"Akron, OH - Market",Outside Account Exec,andreakatusin@iheartmedia.com,Matt Razzante,2007-01-16,"iHeart Media - Akron, OH",Jenny Donheiser,0125G000000uFriQAE
1,iHeart Media,"iHeart Media - Akron, OH",Curtis Qualls,curtisqualls@iheartmedia.com,4.0,7.0,7.0,05/20/2020,10.0,curtisqualls@iheartmedia.com,001f400001KcDFz,003f400001PRfjY,Curtis,Qualls,Curtis Qualls,Cleveland - Area,"Akron, OH - Market",Outside Account Exec,curtisqualls@iheartmedia.com,Jeff Rovinsky,2018-03-30,"iHeart Media - Akron, OH",Jenny Donheiser,0125G000000uFriQAE
2,iHeart Media,"iHeart Media - Akron, OH",David Skoczen,davidskoczen@iheartmedia.com,4.0,9.0,9.0,04/13/2020,10.0,davidskoczen@iheartmedia.com,001f400001KcDFz,003f400001PRfja,David,Skoczen,David Skoczen,Cleveland - Area,"Akron, OH - Market",Outside Account Exec,davidskoczen@iheartmedia.com,Jeff Rovinsky,2018-09-20,"iHeart Media - Akron, OH",Jenny Donheiser,0125G000000uFriQAE
3,iHeart Media,"iHeart Media - Akron, OH",George Allen,georgeallen@iheartmedia.com,4.0,8.0,13.0,04/15/2020,10.0,georgeallen@iheartmedia.com,001f400001KcDG6,003f400001PRflj,George,Allen,George H Allen,Cleveland - Area,"Cleveland, OH - Market",Region SVP Sales,georgeallen@iheartmedia.com,Keith Hotchkiss,1995-07-10,"iHeart Media - Akron, OH",Jenny Donheiser,0125G000000uFriQAE
4,iHeart Media,"iHeart Media - Cleveland, OH",George Allen,georgeallen@iheartmedia.com,4.0,8.0,13.0,04/15/2020,33.0,georgeallen@iheartmedia.com,001f400001KcDG6,003f400001PRflj,George,Allen,George H Allen,Cleveland - Area,"Cleveland, OH - Market",Region SVP Sales,georgeallen@iheartmedia.com,Keith Hotchkiss,1995-07-10,"iHeart Media - Cleveland, OH",Jenny Donheiser,0125G000000uFriQAE
5,iHeart Media,"iHeart Media - Youngstown, OH",George Allen,georgeallen@iheartmedia.com,4.0,8.0,13.0,04/15/2020,6.0,georgeallen@iheartmedia.com,001f400001KcDG6,003f400001PRflj,George,Allen,George H Allen,Cleveland - Area,"Cleveland, OH - Market",Region SVP Sales,georgeallen@iheartmedia.com,Keith Hotchkiss,1995-07-10,"iHeart Media - Youngstown, OH",Jenny Donheiser,0125G000000uFriQAE
6,iHeart Media,"iHeart Media - Akron, OH",Jeff Rovinsky,jeffrovinsky@iheartmedia.com,4.0,0.0,0.0,NaT,10.0,jeffrovinsky@iheartmedia.com,001f400001KcDFz,003f400001PRfjb,Jeff,Rovinsky,Jeff Rovinsky,Cleveland - Area,"Akron, OH - Market",Metro President,jeffrovinsky@iheartmedia.com,Keith Hotchkiss,2014-07-28,"iHeart Media - Akron, OH",Jenny Donheiser,0125G000000uFriQAE
7,iHeart Media,"iHeart Media - Youngstown, OH",Jeff Rovinsky,jeffrovinsky@iheartmedia.com,4.0,0.0,0.0,NaT,6.0,jeffrovinsky@iheartmedia.com,001f400001KcDFz,003f400001PRfjb,Jeff,Rovinsky,Jeff Rovinsky,Cleveland - Area,"Akron, OH - Market",Metro President,jeffrovinsky@iheartmedia.com,Keith Hotchkiss,2014-07-28,"iHeart Media - Youngstown, OH",Jenny Donheiser,0125G000000uFriQAE
8,iHeart Media,"iHeart Media - Akron, OH",Jennifer Grabiec,jennifergrabiec@iheartmedia.com,4.0,7.0,7.0,04/17/2020,10.0,jennifergrabiec@iheartmedia.com,001f400001KcDFz,003f400001PRfjZ,Jennifer,Grabiec,Jennifer Marrie Grabiec,Cleveland - Area,"Akron, OH - Market",Outside Account Exec,jennifergrabiec@iheartmedia.com,Jeff Rovinsky,2015-01-05,"iHeart Media - Akron, OH",Jenny Donheiser,0125G000000uFriQAE
9,iHeart Media,"iHeart Media - Akron, OH",Keith Hotchkiss,keithhotchkiss@iheartmedia.com,4.0,0.0,0.0,NaT,10.0,keithhotchkiss@iheartmedia.com,001f400001KcDG6,003f400001PRflo,Keith,Hotchkiss,Keith Hotchkiss,Cleveland - Area,"Cleveland, OH - Market",Metro President,keithhotchkiss@iheartmedia.com,Tom McConnel

In [ ]:
accounts_ids.drop_duplicates(subset='Account ID',inplace=True)
accounts_ids.to_csv('accounts_ids.csv')

revenue = revenue.drop_duplicates(subset=['iHeart Opportunity ID'])
projects.to_csv('projects_for_salesforce.csv')
revenue.to_csv('revenue_for_salesforce.csv')

In [ ]:
# revenue.loc[(revenue['Form Date'] > '20210402') & (revenue['Form Date'] != 'NaT')]
# projects[projects.apply(lambda row: row.astype(str).str.contains('a0F5G00000MsUHV', case=False).any(), axis=1)]
# newhr[newhr.apply(lambda row: row.astype(str).str.contains('bailey', case=False).any(), axis=1)]
# empl_roll[empl_roll.apply(lambda row: row.astype(str).str.contains('Minneapolis', case=False).any(), axis = 1)]
# user_df[user_df.apply(lambda row: row.astype(str).str.contains('Tallahassee', case=False).any(), axis=1)]

In [ ]:
# # Essentially a VLOOKUP; backfilling Account Name and Revenue Market Area fields with information from newhr because many of these values are consistently absent from
# # the revenue report alone.

# merge_newhr['hr_email'] = merge_newhr.hr_email.str.lower()

# revenue = revenue.merge(merge_newhr[['hr_email', 'hr_market', 'User - Division Parent']], on='email', how='left')
# revenue['Account Name'] = np.where(revenue['Account Name'] == 'iHeart Media - No Data', revenue['User - Division'], revenue['Account Name'])
# revenue['Revenue Market Area'] = np.where(revenue['Revenue Market Area'].isnull(), revenue['User - Division Parent'], revenue['Revenue Market Area'])

# revenue.drop('User - Division', axis=1, inplace=True)
# revenue.drop('User - Division Parent', axis=1, inplace=True)

# print('revenue: ' + str(Differences(revenue['Account Name'].unique().tolist(), accounts.tolist())))
# print(revenue.shape)

In [ ]:
print(revenue.shape)
print(projects.shape)
projects.head(1)
10

In [345]:
projects.project_credits_used = projects.project_credits_used.astype(float)
q1=projects.loc[(projects.project_post_date < '03/31/2021') & (projects.project_post_date > '01/1/2021')].groupby(
    'project_group')[['project_credits_used']].sum().sort_values(by=['project_credits_used'],ascending=False).merge(
    user_df[['group', 'adwire_cap']], left_on='project_group',right_on='group', how='left').drop_duplicates(subset=['group']).rename(
    columns={'project_credits_used':'q1_credits','adwire_cap':'q1_cap'})
q2=projects.loc[(projects.project_post_date < '06/30/2021') & (projects.project_post_date > '03/31/2021')].groupby(
    'project_group')[['project_credits_used']].sum().sort_values(by=['project_credits_used'],ascending=False).merge(
    user_df[['group', 'adwire_cap']], left_on='project_group',right_on='group', how='left').drop_duplicates(subset=['group']).rename(
    columns={'project_credits_used':'q2_credits','adwire_cap':'q2_cap'})
q1_q2 = q1.merge(q2,on='group',how='outer')
q1_q2['percent_used_q1'] = ((q1_q2['q1_credits'])/(q1_q2['q1_cap']*3))
q1_q2['percent_used_q2'] = q1_q2.q2_credits/(q1_q2.q2_cap*3)
q1_q2['percent_change'] = (q1_q2.q2_credits - q1_q2.q1_credits)/(q1_q2.q2_credits*3)
# for i in q1_q2:
#     q1_q2['percent_used_q1'] = q1_q2.q1_credits/(q1_q2.q1_credits*3)
    
q1_q2.to_csv('q1_q2_2021.csv')

In [346]:
# sorted(user_df.sort_values(by='adwire_projects',ascending=False)[:100].group.unique().tolist())

revenue.head()
revenue.sort_values(by='iHeart Gross Revenue',ascending=False)[:100]['Account Name'].unique().tolist()

['iHeart Media - Columbus, OH',
 'iHeart Media - Los Angeles, CA',
 'iHeart Media - New York City, NY',
 nan,
 'iHeart Media - Omaha, NE',
 'iHeart Media - Tampa, FL',
 'iHeart Media - Orlando, FL',
 'iHeart Media - Boston, MA',
 'iHeart Media - Dallas, TX',
 'iHeart Media - Miami, FL',
 'iHeart Media - Phoenix, AZ',
 'iHeart Media - Indianapolis, IN',
 'iHeart Media - Detroit, MI',
 'iHeart Media - San Diego, CA',
 'iHeart Media - Denver, CO',
 'iHeart Media - Sacramento, CA',
 'iHeart Media - Pittsburgh, PA',
 'iHeart Media - Akron, OH',
 'iHeart Media - West Palm Beach, FL',
 'iHeart Media - Washington, DC',
 'iHeart Media - Austin, TX',
 'iHeart Media - Chicago, IL',
 'iHeart Media - Colorado Springs, CO',
 'iHeart Media - Houston, TX',
 'iHeart Media - Grand Rapids, MI',
 'iHeart Media - Oklahoma City, OK',
 'iHeart Media - Madison, WI',
 'iHeart Media - Monterey, CA',
 'iHeart Media - Atlanta, GA',
 'iHeart Media - Albuquerque, NM',
 'iHeart Media - Hartford, CT',
 'iHeart Media 

In [36]:
newusers = pd.merge(empl_roll, user_df, left_on='hr_email', right_on='adwire_email', how='left')
# newusers.loc[newusers.adwire_email.isna()]

In [ ]:
rev_acc_mismatches = revenue.loc[revenue['Account Name'] != revenue['Opportunity: Market: Market Name']]
rev_acc_mismatches = pd.merge(rev_acc_mismatches, projects[['project_id','creator_email','project_group']], on='project_id',how='left')

In [ ]:
rev_acc_mismatches[['Account Name','Opportunity: Market: Market Name','project_group','email','creator_email']].head()

In [43]:
# user_df.head()

In [ ]:
revenue_test = pd.merge(revenue, user_df[['Email','Account ID','name']], left_on='email',right_on='Email',how='left')
revenue_test = pd.merge(revenue, user_df[['Email','Account ID','name']], left_on='email',right_on='Email',how='left')
revenue_test.rename(columns={'Account ID': 'current_account_id'})
revenue_test.head(1)

In [ ]:
start_date = "02/19/2020"
end_date = "04/28/2021"

after_start_date = revenue.form_date >= start_date
before_end_date = revenue.form_date <= end_date
revenue_between = after_start_date & before_end_date
filtered_dates = revenue.loc[revenue_between]

In [ ]:
filtered_dates.shape

In [21]:
results1 = pd.DataFrame(pd.read_csv('Massive Upsert Results 1.csv'))
results1_orig = pd.DataFrame(pd.read_csv('Massive Upsert Results 1.csv'))
results2 = pd.DataFrame(pd.read_csv('Massive Upsert Results 2.csv'))

In [23]:
revenue.form_date.max()

'12/31/2020'

In [23]:
results2.head()

,Id,Success,Created,Error
0,a0F5G00000MsSYPUA3,True,True,NaN
1,a0F5G00000MsSYQUA3,True,True,NaN
2,a0F5G00000MsSYRUA3,True,True,NaN
3,a0F5G00000MsSYSUA3,True,True,NaN
4,a0F5G00000MsSYTUA3,True,True,NaN


In [24]:
results1.Error = [str(x).replace('INVALID_FIELD:Foreign key external ID: ', '') for x in results1.Error]
results1.Error = [str(x).replace(' not found for field iHeart_Opportunity_ID__c in entity Opportunity:--', '') for x in results1.Error]


In [25]:
restults1 = results1.loc[~results1.Error.str.contains('006')]
opp_ids1 = results1.loc[results1.Error.str.contains('006')]
opp_ids1.to_csv('results_error_opportunities_1.csv')

In [26]:
restults1.Error =[str(x)[:50] for x in restults1.Error]
restults1.Error.unique()

/Users/arlisscoates/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


array(['nan', '6123456789012',
       'FIELD_INTEGRITY_EXCEPTION:Specified External ID fo',
       'DUPLICATE_VALUE:Duplicate external id specified: B',
       'DUPLICATE_VALUE:Duplicate external id specified: E',
       'DUPLICATE_VALUE:Duplicate external id specified: V',
       'DUPLICATE_VALUE:Duplicate external id specified: S',
       'DUPLICATE_VALUE:Duplicate external id specified: C',
       'DUPLICATE_VALUE:Duplicate external id specified: A',
       'DUPLICATE_VALUE:Duplicate external id specified: G',
       'DUPLICATE_VALUE:Duplicate external id specified: D',
       'DUPLICATE_VALUE:Duplicate external id specified: T',
       'DUPLICATE_VALUE:Duplicate external id specified: M',
       'DUPLICATE_VALUE:Duplicate external id specified: H',
       'DUPLICATE_VALUE:Duplicate external id specified: L',
       'DUPLICATE_VALUE:Duplicate external id specified: 6',
       'DUPLICATE_VALUE:Duplicate external id specified: F',
       'DUPLICATE_VALUE:Duplicate external id specifie

In [27]:
results

NameError: name 'results' is not defined

In [28]:
results2.Error = [str(x).replace('INVALID_FIELD:Foreign key external ID: ', '') for x in results2.Error]
results2.Error = [str(x).replace(' not found for field iHeart_Opportunity_ID__c in entity Opportunity:--', '') for x in results2.Error]

In [29]:
matched = pd.DataFrame(pd.read_csv('report1619705426780 - Sortable.csv'))
matched.dropna(inplace=True)
matched.rename(columns={"AdWire Project ID's": 'project_id'}, inplace=True)
matched_revenue = matched.merge(revenue, on='project_id', how='left')

In [30]:
matched_revenue.to_csv('matched_revenue.csv')

In [43]:
# results2.Error.unique().tolist()

In [32]:
subprojects.head()

NameError: name 'subprojects' is not defined

In [33]:
subprojects = projects.loc[(projects.project_post_date >= '03/01/2021') & (projects.project_post_date <= '05/31/2021')]

In [34]:
counts = pd.DataFrame(data=projects.project_group.value_counts())
counts.index.name = 'group'
counts.rename(columns={'project_group': 'count'},inplace=True)

In [35]:
counts = pd.merge(counts,subprojects[['project_group','adwire_cap']], left_on='group', right_on = 'project_group', how='left')

In [36]:
# pd.merge(subprojects, user_df)
counts.head()

,count,project_group,adwire_cap
0,899,iHeart Media - Inside Sales,80.0
1,899,iHeart Media - Inside Sales,80.0
2,899,iHeart Media - Inside Sales,11.0
3,899,iHeart Media - Inside Sales,11.0
4,899,iHeart Media - Inside Sales,11.0


In [37]:
# counts.sort_values(['count','adwire_cap'], ascending = ('True','True'))

In [38]:
# counts.drop_duplicates(subset=['project_group'])drop_duplicates

In [39]:
revenue.head()

,iHeart Opportunity ID,Stage,Revenue Market Division,Revenue Market Area,Account Name,iHeart Oppty Owner,email,new_business,opp_account_name,Opportunity Name,project_id,form_date,Created Date,Close Date,Expected iHeart Revenue,iHeart Gross Revenue,Created on Closed Opp,Opportunity: Market: Market Name,Revenue Market Type,Email,Account ID,wl_owner
0,0060y00001Fo10M,Closed - Won,NaN,Inside Sales & Business Development,iHeart Media - Inside Sales,James Boyer,jamesboyer@iheartmedia.com,New Business,Nymbl Science,Nymbl Scie-Schroeder-Inside Sales-9399588,B9ZgHBDwAAA,03/02/2021,3/1/2021,3/3/2021,3402.00,3402.00,0,iHeart Media - No Data,NaN,jamesboyer@iheartmedia.com,0015G00001bTPub,Ashley McCarthy
1,0060y00001D17Qo,Closed - Won,NaN,Inside Sales & Business Development,iHeart Media - Inside Sales,Scott Schoenberg,scottschoenberg@iheartmedia.com,Existing Business,Woolerbrands,Woolerbran-MMP-34544_Woolerbrands_Howler Head ...,A74MmDowAAA,06/30/2020,1/30/2020,7/15/2020,15000.00,15000.00,0,"iHeart Media - Fort Myers/Naples, FL",NaN,scottschoenberg@iheartmedia.com,0015G00001bTPub,Ashley McCarthy
2,0060y00001EdPc4,Closed - Won,NaN,NaN,NaN,Lauren Annucci,laurenannucci@iheartmedia.com,Existing Business,Save Florida Jobs Inc.,Save Our J-Vote No on Amendment 2-MMS-8621600,BMVVeECQAAA,08/19/2020,8/13/2020,9/9/2020,88235.29,88235.29,0,"iHeart Media - Tampa, FL",NaN,NaN,NaN,0
3,0060y00001BJfCT,Closed - Won,NaN,Inside Sales & Business Development,iHeart Media - Inside Sales,Scott Schoenberg,scottschoenberg@iheartmedia.com,Existing Business,Vita Coco/All Market Inc.,MMP-31480_VitaCoco_Tampa_Houston-Inside Sales-...,AVCPoABgAAA,03/04/2020,12/6/2019,1/24/2020,147840.00,147840.00,1,"iHeart Media - Los Angeles, CA",NaN,scottschoenberg@iheartmedia.com,0015G00001bTPub,Ashley McCarthy
4,0060y00001D17Qo,Closed - Won,NaN,Inside Sales & Business Development,iHeart Media - Inside Sales,Scott Schoenberg,scottschoenberg@iheartmedia.com,Existing Business,Woolerbrands,Woolerbran-MMP-34544_Woolerbrands_Howler Head ...,A74JiBEQAAA,06/30/2020,1/30/2020,7/15/2020,15000.00,15000.00,0,"iHeart Media - Fort Myers/Naples, FL",NaN,scottschoenberg@iheartmedia.com,0015G00001bTPub,Ashley McCarthy


In [40]:
revenue_range = revenue.loc[(revenue.form_date > '03/01/2021') & (revenue.form_date < '05/31/2021')]

In [41]:
revenue_range.shape

(6224, 22)

In [42]:
revenue.loc[(revenue.form_date > '03/01/2021') & (rvenue.form_date < '05/31/2021')].shape

NameError: name 'rvenue' is not defined

In [44]:
markets_spend.head()

,client,group,projects,spend,project_ts,group_cap
0,iHeart Media,"Tampa, FL",44,46.5,2018-10-23 14:55:53,35
1,iHeart Media,"Tampa, FL",35,41.0,2018-12-11 15:10:31,35
2,iHeart Media,"Tampa, FL",10,15.5,2018-10-29 18:32:36,35
3,iHeart Media,"Tampa, FL",22,25.0,2018-10-24 15:54:31,35
4,iHeart Media,"Tampa, FL",42,48.0,2018-10-26 12:58:28,35


In [ ]:
markets_spend.drop_duplicates(by='')